## Calculate the ZA transport tensor

Here we take the time averaged fields, averaged on model Zgrid, and estiamte the corresponding transport tensor. 

In [1]:
import numpy as np
import xarray as xr
#from funcs import *
%matplotlib inline
from matplotlib import pyplot as plt
from matplotlib import colors, ticker, cm
from matplotlib.colors import LogNorm

In [2]:
from xgcm import Grid
from scipy.linalg import pinv, eig, eigh


In [3]:
Model_Tav = xr.open_dataset('./analysis_data_files/ZA_model_Tav.nc')
Trac_Tav  = xr.open_dataset('./analysis_data_files/ZA_Trac_Tav.nc')

In [4]:
grid = Grid(Model_Tav, periodic='X')

In [5]:
list_trac = ['TRAC01', 'TRAC02', 'TRAC03', 'TRAC04', 'TRAC05', 
            'TRAC06', 'TRAC07', 'TRAC08', 'TRAC09', 'TRAC10', 
            'TRAC11', 'TRAC12', 'TRAC13', 'TRAC14', 'TRAC15', 
            'TRAC16', 'TRAC17', 'TRAC18', 'TRAC19', 'TRAC20']

In [6]:
# Make a single dataset with all the gradients 
ds_grads = xr.Dataset()

dx = 5e3 # model resolution 

for var_name in list_trac: 
    
    ds_grads['d'+var_name+'dx'] = grid.interp(grid.diff(Trac_Tav[var_name], 'X')/dx, 'X')
    ds_grads['d'+var_name+'dy'] = grid.interp(grid.diff(Trac_Tav[var_name], 'Y', boundary='extend')/dx,
                                              'Y', boundary='extend')
    ds_grads['d'+var_name+'dz'] = grid.interp(grid.diff(Trac_Tav[var_name], 'Z', boundary='extend')/
                                              grid.diff(Trac_Tav['Z'], 'Z', boundary='extend'), 
                                              'Z', boundary='extend')

In [7]:
# Make a single dataset with the eddy fluxes (upcp)
ds_eddy = xr.Dataset()

for var_name in list_trac:
    
    ds_eddy['Up'+var_name+'p'] = (grid.interp(Trac_Tav['U'+var_name],'X') - 
                                  grid.interp(Model_Tav.uVeltave,'X')*Trac_Tav[var_name])
    ds_eddy['Vp'+var_name+'p'] = (grid.interp(Trac_Tav['V'+var_name], 'Y', boundary='extend') - 
                                  grid.interp(Model_Tav.vVeltave,'Y', boundary='extend')*Trac_Tav[var_name])
    ds_eddy['Wp'+var_name+'p'] = (Trac_Tav['W'+var_name] - 
                                  grid.interp(Model_Tav.wVeltave,'Z', boundary='extend')*Trac_Tav[var_name])
    

In [8]:
def get_flux_arrays(ds, list_tracers): 
    # U'C'
    testxr1 = ds['UpTRAC01p']
    testxr1['tracer_num'] = 1

    UpCp = testxr1 

    n=2
    for i in list_tracers[1:]: 
        temp = ds['Up'+i+'p']
        temp['tracer_num'] = n 
        n=n+1
        UpCp = xr.concat([UpCp, temp], dim='tracer_num')
    
    UpCp.name = 'UpCp'
    
    # V'C'
    testxr1 = ds['VpTRAC01p']
    testxr1['tracer_num'] = 1

    VpCp = testxr1 

    n=2
    for i in list_tracers[1:]: 
        temp = ds['Vp'+i+'p']
        temp['tracer_num'] = n 
        n=n+1

        VpCp = xr.concat([VpCp, temp], dim='tracer_num')

    VpCp.name = 'VpCp'
    
    # W'C'
    testxr1 = ds['WpTRAC01p']
    testxr1['tracer_num'] = 1

    WpCp = testxr1 

    n=2
    for i in list_tracers[1:]: 
        temp = ds['Wp'+i+'p']
        temp['tracer_num'] = n 
        n=n+1

        WpCp = xr.concat([WpCp, temp], dim='tracer_num')
    
    WpCp.name = 'WpCp'
    
    return [UpCp, VpCp, WpCp]

In [9]:
[UpCp, VpCp, WpCp] = get_flux_arrays(ds_eddy, list_trac)

In [10]:
def get_grad_arrays(ds, list_tracers): 
    # Put tracer gradients into xarrays
# dCdx
    testxr1 = ds['dTRAC01dx']
    testxr1['tracer_num'] = 1

    dCdx = testxr1 

    n=2
    for i in list_tracers[1:]: 
        temp = ds['d'+i+'dx']
        temp['tracer_num'] = n 
        n=n+1

        dCdx = xr.concat([dCdx, temp], dim='tracer_num')
    
    dCdx.name = 'dCdx'
    
    # dCdy
    testxr1 = ds['dTRAC01dy']
    testxr1['tracer_num'] = 1

    dCdy = testxr1 

    n=2
    for i in list_tracers[1:]: 
        temp = ds['d'+i+'dy']
        temp['tracer_num'] = n 
        n=n+1

        dCdy = xr.concat([dCdy, temp], dim='tracer_num')   
    dCdy.name = 'dCdy'
    
    # dCdz
    testxr1 = ds['dTRAC01dz']
    testxr1['tracer_num'] = 1

    dCdz = testxr1 

    n=2
    for i in list_tracers[1:]: 
        temp = ds['d'+i+'dz']
        temp['tracer_num'] = n 
        n=n+1

        dCdz = xr.concat([dCdz, temp], dim='tracer_num')
    dCdz.name = 'dCdz'
    
    return [dCdx, dCdy, dCdz]

In [11]:
[dCdx, dCdy, dCdz]= get_grad_arrays(ds_grads, list_trac)

In [34]:
# coarsen fields (in older codes this wa done using the spatial_average notebooks)
npts = int(50e3/dx)
UpCp_coarse = UpCp.coarsen(XC=npts, YC=npts).mean().rename('UpCp')
VpCp_coarse = VpCp.coarsen(XC=npts, YC=npts).mean().rename('VpCp')
WpCp_coarse = WpCp.coarsen(XC=npts, YC=npts).mean().rename('WpCp')

dCdx_coarse = dCdx.coarsen(XC=npts, YC=npts).mean().rename('dCdx')
dCdy_coarse = dCdy.coarsen(XC=npts, YC=npts).mean().rename('dCdy')
dCdz_coarse = dCdz.coarsen(XC=npts, YC=npts).mean().rename('dCdz')

In [36]:
UpCp_coarse.to_netcdf('analysis_data_files/UpCp_ZA_50km.nc')
VpCp_coarse.to_netcdf('analysis_data_files/VpCp_ZA_50km.nc')
WpCp_coarse.to_netcdf('analysis_data_files/WpCp_ZA_50km.nc')

dCdx_coarse.to_netcdf('analysis_data_files/dCdx_ZA_50km.nc')
dCdy_coarse.to_netcdf('analysis_data_files/dCdy_ZA_50km.nc')
dCdz_coarse.to_netcdf('analysis_data_files/dCdz_ZA_50km.nc')

/home/dbalwada/.conda/envs/pangeo/lib/python3.7/site-packages/xarray/core/dataarray.py:2664: SerializationWarning: saving variable maskC with floating point data as an integer dtype without any _FillValue to use for NaNs
  return dataset.to_netcdf(*args, **kwargs)


In [13]:
# function to estimate the diffusivity tensor
def calc_tensor(uc,vc,wc, cx,cy,cz):
    Aflux = np.array([uc, vc, wc])
    Agrad = np.array([cx, cy, cz])

    if ~(np.isnan(Agrad).any() | np.isnan(Aflux).any()):
        return -(Aflux.dot(pinv(Agrad)))
    else:
        return np.nan*(Aflux.dot(Agrad.T))

In [37]:
UpCp_coarse

<xarray.DataArray 'UpCp' (tracer_num: 20, Z: 40, YC: 40, XC: 40)>
array([[[[ 1.39918027e-03,  1.61405723e-03,  1.75714493e-03, ...,
           1.73145381e-03,  1.56505872e-03,  1.41407165e-03],
         [ 1.30959542e-03,  1.14825403e-03,  1.33495440e-03, ...,
           1.24135171e-03,  1.33382680e-03,  1.41093833e-03],
         [ 7.30100437e-04,  6.18715538e-04,  5.61708584e-04, ...,
           9.90016386e-04,  1.09324744e-03,  8.29368131e-04],
         ...,
         [-1.00846228e-03, -1.14357995e-03, -1.10708654e-03, ...,
          -9.69166926e-04, -8.36756954e-04, -8.60186236e-04],
         [-1.16399128e-03, -1.31041731e-03, -1.26298692e-03, ...,
          -8.47707270e-04, -8.51823308e-04, -9.69635614e-04],
         [-1.15040434e-03, -1.06766168e-03, -9.48332250e-04, ...,
          -1.37390580e-03, -1.25946151e-03, -1.23684446e-03]],

        [[ 1.38981338e-03,  1.60546973e-03,  1.74742355e-03, ...,
           1.71837653e-03,  1.55303732e-03,  1.40438532e-03],
         [ 1.29387795e-03,  1.13633915e-03,  1.32235559e-03, ...,
           1.22237939e-03,  1.31470500e-03,  1.39378791e-03],
         [ 7.16116570e-04,  6.08286064e-04,  5.51887206e-04, ...,
           9.69041837e-04,  1.07396685e-03,  8.12253274e-04],
...
           1.97484449e-04,  1.63650402e-04,  1.63108183e-04],
         [ 1.13808012e-04,  9.77543532e-05,  9.00660089e-05, ...,
           8.57131163e-05,  8.27280746e-05,  9.39197926e-05],
         [-9.32764262e-04, -8.75263417e-04, -8.90099909e-04, ...,
          -8.22858361e-04, -9.54383111e-04, -9.66363936e-04]],

        [[ 6.94671937e-04,  7.76511442e-04,  8.03430506e-04, ...,
           6.42852334e-04,  6.31170289e-04,  6.59078185e-04],
         [ 1.44546531e-04,  1.52998473e-04,  1.67378559e-04, ...,
           1.28049738e-04,  1.10400033e-04,  1.25411680e-04],
         [ 7.96224413e-06,  3.57272461e-06, -4.66345045e-06, ...,
          -1.07739788e-05, -2.39501223e-05, -2.24832220e-05],
         ...,
         [ 1.29776585e-04,  1.16545467e-04,  1.12086367e-04, ...,
           2.20444534e-04,  1.41735407e-04,  1.51736065e-04],
         [-1.04379767e-04, -1.86633435e-04, -1.98706708e-04, ...,
          -4.97013061e-05, -8.87487549e-05, -1.05906322e-04],
         [-1.62105320e-03, -1.49287377e-03, -1.48690445e-03, ...,
          -1.73145707e-03, -1.79152295e-03, -1.74053316e-03]]]],
      dtype=float32)
Coordinates:
  * Z           (Z) float32 -5.0 -15.0 -25.0 ... -2.728e+03 -2.83e+03 -2.934e+03
  * YC          (YC) float32 2.5e+04 7.5e+04 1.25e+05 ... 1.925e+06 1.975e+06
  * XC          (XC) float32 2.5e+04 7.5e+04 1.25e+05 ... 1.925e+06 1.975e+06
    rA          (YC, XC) float32 2.5e+07 2.5e+07 2.5e+07 ... 2.5e+07 2.5e+07
    Depth       (YC, XC) float32 2.686e+03 2.686e+03 ... 2.985e+03 2.985e+03
    drF         (Z) float32 10.0 10.0 10.0 12.0 14.0 ... 103.0 103.0 103.0 103.0
    PHrefC      (Z) float32 49.05 147.1 245.2 ... 2.676e+04 2.777e+04 2.878e+04
    hFacC       (Z, YC, XC) float32 0.9 0.9 0.9 0.9 0.9 ... 1.0 1.0 1.0 1.0 1.0
    maskC       (Z, YC, XC) float64 0.9 0.9 0.9 0.9 0.9 ... 1.0 1.0 1.0 1.0 1.0
  * tracer_num  (tracer_num) int64 1 2 3 4 5 6 7 8 9 ... 13 14 15 16 17 18 19 20

In [14]:
%%time
# Calculate the tensor
# This is slow (can take over an hour)

Ktensor_fast = xr.apply_ufunc(calc_tensor, 
                       UpCp_coarse.sel(tracer_num=slice(1,19,2)),
                       VpCp_coarse.sel(tracer_num=slice(1,19,2)),
                       WpCp_coarse.sel(tracer_num=slice(1,19,2)),
                       dCdx_coarse.sel(tracer_num=slice(1,19,2)),
                       dCdy_coarse.sel(tracer_num=slice(1,19,2)),
                       dCdz_coarse.sel(tracer_num=slice(1,19,2)),
                       input_core_dims=[['tracer_num'], ['tracer_num'], ['tracer_num'], ['tracer_num'], ['tracer_num'], ['tracer_num']],
                       vectorize=True, output_core_dims=[['i','j']], dask='parallelized', 
                       output_dtypes=['float32'], output_sizes={'i':3,'j':3})

Ktensor_fast.load() # need to load because we will take transpose and a

/home/dbalwada/.conda/envs/pangeo/lib/python3.7/site-packages/ipykernel/__main__.py:13: FutureWarning: ``output_sizes`` should be given in the ``dask_gufunc_kwargs`` parameter. It will be removed as direct parameter in a future version.


CPU times: user 9.5 s, sys: 52.8 ms, total: 9.55 s
Wall time: 9.55 s


<xarray.DataArray (Z: 40, YC: 40, XC: 40, i: 3, j: 3)>
array([[[[[            nan,             nan,             nan],
          [            nan,             nan,             nan],
          [            nan,             nan,             nan]],

         [[            nan,             nan,             nan],
          [            nan,             nan,             nan],
          [            nan,             nan,             nan]],

         [[            nan,             nan,             nan],
          [            nan,             nan,             nan],
          [            nan,             nan,             nan]],

         ...,

         [[            nan,             nan,             nan],
          [            nan,             nan,             nan],
          [            nan,             nan,             nan]],

         [[            nan,             nan,             nan],
          [            nan,             nan,             nan],
...
          [-1.62283480e-01, -9.10187960e-02,  3.94871517e-04]],

         [[ 9.15073145e+03, -1.89523026e+02, -7.05742359e-01],
          [-1.16339317e+02,  1.88908035e+02, -4.83532548e-02],
          [-2.63258845e-01, -5.64191565e-02,  3.88822576e-04]],

         ...,

         [[ 8.06050342e+03, -1.46148178e+02, -1.70140743e-01],
          [-4.27703827e+02,  2.71697968e+02, -1.85430408e-01],
          [-1.65017292e-01, -1.39899507e-01,  5.24993928e-04]],

         [[ 6.77281201e+03, -4.50062408e+02, -7.35439777e-01],
          [-1.97837585e+02,  3.25690460e+02, -1.68033659e-01],
          [-1.12095386e-01, -1.33742735e-01,  5.07759571e-04]],

         [[ 7.89883691e+03, -1.64723892e+01, -1.27865934e+00],
          [-1.45799042e+02,  2.89942719e+02, -6.08886480e-02],
          [-1.12333253e-01, -1.22418560e-01,  4.66534053e-04]]]]],
      dtype=float32)
Coordinates:
  * Z        (Z) float32 -5.0 -15.0 -25.0 ... -2.728e+03 -2.83e+03 -2.934e+03
  * YC       (YC) float32 2.5e+04 7.5e+04 1.25e+05 ... 1.925e+06 1.975e+06
  * XC       (XC) float32 2.5e+04 7.5e+04 1.25e+05 ... 1.925e+06 1.975e+06
    rA       (YC, XC) float32 2.5e+07 2.5e+07 2.5e+07 ... 2.5e+07 2.5e+07
    Depth    (YC, XC) float32 2.686e+03 2.686e+03 ... 2.985e+03 2.985e+03
    drF      (Z) float32 10.0 10.0 10.0 12.0 14.0 ... 103.0 103.0 103.0 103.0
    PHrefC   (Z) float32 49.05 147.1 245.2 ... 2.676e+04 2.777e+04 2.878e+04
    hFacC    (Z, YC, XC) float32 0.9 0.9 0.9 0.9 0.9 0.9 ... 1.0 1.0 1.0 1.0 1.0
    maskC    (Z, YC, XC) float64 0.9 0.9 0.9 0.9 0.9 0.9 ... 1.0 1.0 1.0 1.0 1.0
Dimensions without coordinates: i, j

In [15]:
%%time
# Calculate the tensor
# This is slow (can take over an hour)

Ktensor_slow = xr.apply_ufunc(calc_tensor, 
                       UpCp_coarse.sel(tracer_num=slice(2,20,2)),
                       VpCp_coarse.sel(tracer_num=slice(2,20,2)),
                       WpCp_coarse.sel(tracer_num=slice(2,20,2)),
                       dCdx_coarse.sel(tracer_num=slice(2,20,2)),
                       dCdy_coarse.sel(tracer_num=slice(2,20,2)),
                       dCdz_coarse.sel(tracer_num=slice(2,20,2)),
                       input_core_dims=[['tracer_num'], ['tracer_num'], ['tracer_num'], ['tracer_num'], ['tracer_num'], ['tracer_num']],
                       vectorize=True, output_core_dims=[['i','j']], dask='parallelized', 
                       output_dtypes=['float32'], output_sizes={'i':3,'j':3})

Ktensor_slow.load() # need to load because we will take transpose and a

/home/dbalwada/.conda/envs/pangeo/lib/python3.7/site-packages/ipykernel/__main__.py:13: FutureWarning: ``output_sizes`` should be given in the ``dask_gufunc_kwargs`` parameter. It will be removed as direct parameter in a future version.


CPU times: user 8.66 s, sys: 182 ms, total: 8.84 s
Wall time: 8.83 s


<xarray.DataArray (Z: 40, YC: 40, XC: 40, i: 3, j: 3)>
array([[[[[            nan,             nan,             nan],
          [            nan,             nan,             nan],
          [            nan,             nan,             nan]],

         [[            nan,             nan,             nan],
          [            nan,             nan,             nan],
          [            nan,             nan,             nan]],

         [[            nan,             nan,             nan],
          [            nan,             nan,             nan],
          [            nan,             nan,             nan]],

         ...,

         [[            nan,             nan,             nan],
          [            nan,             nan,             nan],
          [            nan,             nan,             nan]],

         [[            nan,             nan,             nan],
          [            nan,             nan,             nan],
...
          [-1.55109897e-01, -9.73459855e-02,  4.68396116e-04]],

         [[ 1.00719492e+04, -2.61940186e+02,  4.65407372e-02],
          [-8.20117645e+01,  1.79253052e+02,  8.50999355e-03],
          [-2.91073680e-01, -6.19485117e-02,  4.15037386e-04]],

         ...,

         [[ 9.77714355e+03,  2.63049469e+02, -3.59535217e-02],
          [-4.30071442e+02,  2.50729507e+02, -1.66520178e-01],
          [-1.90596536e-01, -1.49540827e-01,  5.74943202e-04]],

         [[ 7.90922070e+03, -3.22197571e+02, -2.52686501e-01],
          [-1.77086761e+02,  3.01692078e+02, -1.83602810e-01],
          [-1.08315788e-01, -1.36861369e-01,  5.58800879e-04]],

         [[ 9.36535156e+03,  7.49930267e+01, -1.04073358e+00],
          [-1.48106537e+02,  2.60533875e+02, -6.19322062e-02],
          [-1.10944852e-01, -1.29959598e-01,  5.37640939e-04]]]]],
      dtype=float32)
Coordinates:
  * Z        (Z) float32 -5.0 -15.0 -25.0 ... -2.728e+03 -2.83e+03 -2.934e+03
  * YC       (YC) float32 2.5e+04 7.5e+04 1.25e+05 ... 1.925e+06 1.975e+06
  * XC       (XC) float32 2.5e+04 7.5e+04 1.25e+05 ... 1.925e+06 1.975e+06
    rA       (YC, XC) float32 2.5e+07 2.5e+07 2.5e+07 ... 2.5e+07 2.5e+07
    Depth    (YC, XC) float32 2.686e+03 2.686e+03 ... 2.985e+03 2.985e+03
    drF      (Z) float32 10.0 10.0 10.0 12.0 14.0 ... 103.0 103.0 103.0 103.0
    PHrefC   (Z) float32 49.05 147.1 245.2 ... 2.676e+04 2.777e+04 2.878e+04
    hFacC    (Z, YC, XC) float32 0.9 0.9 0.9 0.9 0.9 0.9 ... 1.0 1.0 1.0 1.0 1.0
    maskC    (Z, YC, XC) float64 0.9 0.9 0.9 0.9 0.9 0.9 ... 1.0 1.0 1.0 1.0 1.0
Dimensions without coordinates: i, j

In [16]:
lam1 = 1/31104000.
lam2 = 1/186624000.

Ktensor_corr = (lam2*Ktensor_fast - lam1*Ktensor_slow)/(lam2 - lam1)


In [18]:
Ktensor_corrT = Ktensor_corr.transpose('Z','YC','XC','j','i')

STcorr = 0.5*(Ktensor_corr.data + Ktensor_corrT.data)
ATcorr = 0.5*(Ktensor_corr.data - Ktensor_corrT.data)

STcorr= xr.DataArray(STcorr, coords=Ktensor_corr.coords, dims=Ktensor_corr.dims)
ATcorr = xr.DataArray(ATcorr, coords=Ktensor_corr.coords, dims=Ktensor_corr.dims)

In [25]:
# make sure the eigen values are arranged by magnitude (instead of the default arrangement)

def eigen(A):
    if np.isnan(A).any(): A = np.nan_to_num(A)
    eigenValues, eigenVectors = eigh(A)
    idx = np.argsort(np.abs(eigenValues))
    eigenValues = eigenValues[idx]
    eigenVectors = eigenVectors[:,idx]
    return (eigenValues, eigenVectors)

In [26]:
# calculate the eigenvalues and eigenvectors of symmetric part
eigvalsSTcorr, eigvecsSTcorr = xr.apply_ufunc(eigen, STcorr, input_core_dims=[['i','j']],
                                    vectorize=True, output_core_dims=[['ii'], ['k','ii']])

In [29]:
diff_tensor = xr.Dataset()
diff_tensor['Kfast'] = Ktensor_fast
diff_tensor['Kslow'] = Ktensor_slow
diff_tensor['Kcorr'] = Ktensor_corr
diff_tensor['STcorr'] = STcorr
diff_tensor['ATcorr'] = ATcorr
diff_tensor['eigvalsSTcorr'] = eigvalsSTcorr
diff_tensor['eigvecsSTcorr'] = eigvecsSTcorr

In [31]:
diff_tensor

<xarray.Dataset>
Dimensions:        (XC: 40, YC: 40, Z: 40, i: 3, ii: 3, j: 3, k: 3)
Coordinates:
  * Z              (Z) float32 -5.0 -15.0 -25.0 ... -2.83e+03 -2.934e+03
  * YC             (YC) float32 2.5e+04 7.5e+04 1.25e+05 ... 1.925e+06 1.975e+06
  * XC             (XC) float32 2.5e+04 7.5e+04 1.25e+05 ... 1.925e+06 1.975e+06
    rA             (YC, XC) float32 2.5e+07 2.5e+07 2.5e+07 ... 2.5e+07 2.5e+07
    Depth          (YC, XC) float32 2.686e+03 2.686e+03 ... 2.985e+03 2.985e+03
    drF            (Z) float32 10.0 10.0 10.0 12.0 ... 103.0 103.0 103.0 103.0
    PHrefC         (Z) float32 49.05 147.1 245.2 ... 2.777e+04 2.878e+04
    hFacC          (Z, YC, XC) float32 0.9 0.9 0.9 0.9 0.9 ... 1.0 1.0 1.0 1.0
    maskC          (Z, YC, XC) float64 0.9 0.9 0.9 0.9 0.9 ... 1.0 1.0 1.0 1.0
Dimensions without coordinates: i, ii, j, k
Data variables:
    Kfast          (Z, YC, XC, i, j) float32 nan nan nan ... -0.1224 0.0004665
    Kslow          (Z, YC, XC, i, j) float32 nan nan nan ... -0.13 0.0005376
    Kcorr          (Z, YC, XC, i, j) float32 nan nan nan ... -0.1315 0.0005519
    STcorr         (Z, YC, XC, i, j) float32 nan nan nan ... -0.0968 0.0005519
    ATcorr         (Z, YC, XC, i, j) float32 nan nan nan ... 0.4412 -0.03466 0.0
    eigvalsSTcorr  (Z, YC, XC, ii) float32 0.0 0.0 0.0 ... 254.6 9.659e+03
    eigvecsSTcorr  (Z, YC, XC, k, ii) float32 1.0 0.0 ... 0.0003866 -5.711e-05

In [30]:
diff_tensor.to_netcdf('analysis_data_files/diff_tensor_ZA_50km.nc')

/home/dbalwada/.conda/envs/pangeo/lib/python3.7/site-packages/ipykernel/__main__.py:1: SerializationWarning: saving variable maskC with floating point data as an integer dtype without any _FillValue to use for NaNs
  if __name__ == '__main__':
